In [1]:
# def pso():
#     swarm = [Particle...]
#     while not stop_condition():
#         for p in swarm:
#             # v = c_i*inertia + c_p*personal_best + c_s*swarm_best
#             p.update_velocity()
#             p.update_position()
#     return najbolja pozicija za celo jato

In [2]:
import numpy as np
def rastrigin(x):
    A = 10
    n = len(x)
    return A*n + sum(x[i]**2 - A*np.cos(2*np.pi*x[i]) for i in range(n))

In [3]:
num_dimensions = 2
bounds = np.array([(-5.12, 5.12) for _ in range(num_dimensions)])

In [4]:
lower_bounds = bounds[:,0]

In [5]:
class Particle:
    swarm_best_position = None
    swarm_best_value = None
    
    def __init__(self, bounds, f, c_i, c_p, c_s):
        self.c_i = c_i
        self.c_p = c_p
        self.c_s = c_s
        
        self.lower_bounds = bounds[:,0]
        self.upper_bounds = bounds[:,1]
        self.position = np.random.uniform(self.lower_bounds, self.upper_bounds, len(bounds))
        self.velocity = np.random.uniform(-1, 1, len(bounds))
        
        self.personal_best_position = self.position.copy()
        self.value = f(self.position)
        self.personal_best_value = self.value
        self.f = f
        if Particle.swarm_best_value is None or self.value < Particle.swarm_best_value:
            Particle.swarm_best_value = self.value
            Particle.swarm_best_position = self.position.copy()
            
    def update_position(self):
        self.position = np.clip(self.position + self.velocity, 
                               self.lower_bounds,
                               self.upper_bounds)
        self.value = self.f(self.position)
        if self.value < self.personal_best_value:
            self.personal_best_value = self.value
            self.personal_best_position = self.position.copy()
            if self.value < Particle.swarm_best_value:
                Particle.swarm_best_value = self.value
                Particle.swarm_best_position = self.position.copy()
    
    def update_velocity(self): # za domaci arg iter - c_p se smanjuje, a c_s se povecava
        cognitive_velocity = self.personal_best_position - self.position
        social_velocity = Particle.swarm_best_position - self.position
        r_p = np.random.random(len(cognitive_velocity))
        r_s = np.random.random(len(social_velocity))
        self.velocity = (self.c_i * self.velocity
                        + self.c_p * r_p * cognitive_velocity
                        + self.c_s * r_s * social_velocity)

update_velocity:
$$vel_{cog} = (pbp - pos)$$
$$vel_{soc} = (gbp - pos)$$
$$velocity = c_i * velocity + r_p * c_p * vel_{cog} +  r_s * c_s * vel_{soc}$$

In [6]:
def pso(swarm_size, num_iters):
    swarm = [Particle(bounds, f=rastrigin, c_i=0.5, c_p=1, c_s=1) for _ in range(swarm_size)]
    for i in range(num_iters):
        for p in swarm:
            p.update_velocity()
            p.update_position()
    return Particle.swarm_best_position, Particle.swarm_best_value

In [7]:
pso(swarm_size=20, num_iters=100)

(array([-1.24192230e-09,  3.42254228e-09]), 0.0)